In [1]:
import ROOT as r
import numpy as np

Welcome to JupyROOT 6.22/06


In [2]:
def thetaPhi():
    '''returns random theta and phi values'''
    theta = np.random.uniform(0,2*np.pi)
    phi = np.random.uniform(0,np.pi)
    return theta, phi

def momenta(mag, theta, phi):
    '''given an input magnitude, theta and phi the cartesian momenta are returned'''
    Px = mag * np.sin(theta) * np.cos(phi) 
    Py = mag * np.sin(theta) * np.sin(phi)
    Pz = mag * np.cos(theta)
    
    return Px, Py, Pz

def vector(g, theta, phi, mag, mass):
    '''given a tlorentzvector g the cartesian components are set with the circular components and the mass'''
    Px, Py, Pz = momenta(mag, theta, phi)
    g.SetPxPyPzE(Px, Py, Pz, mass)
    
    
def boost(px, py, pz, e, beta):
    '''input 4 momenta of particle wanted to be boosted, and beta the ratio v/c which it is boosted by
    returns the new boosted 4 momenta'''
    #initialising arrays for boosted momenta
    boostx = np.zeros([len(px)])
    boosty = np.zeros([len(px)])
    boostz = np.zeros([len(px)])
    booste = np.zeros([len(px)])
    
    #looping over momenta values   
    for i in range(len(px)):
        vec = r.Math.LorentzVector('ROOT::Math::PxPyPzE4D<double>')(px[i], py[i], pz[i], e[i]) #using this type of lorentzvec works better
        boost = r.Math.VectorUtil.boostZ(vec, beta) # boosting by factor beta
        #components from the boosted lorentzvector are appended to arrays for momenta
        boostx[i], boosty[i], boostz[i], booste[i] = boost.Px(), boost.Py(), boost.Pz(), boost.E()
        
        
    return boostx,boosty,boostz,booste

def wArr(num):
    '''just for making numpy array of stationary w given the number of events'''
    momenta = np.zeros([num,4])
    momenta[:,3] = wMass
    
    return momenta

In [3]:
wMass = 80.3

eMass = 0.0005

nMass = 0

# equations for this bit found here: http://www.phys.ufl.edu/~avery/course/4390/f2015/lectures/relativistic_kinematics_2.pdf

eE = (wMass**2 + eMass**2)/(2*wMass) # energy of the electron

mag = np.sqrt(eE**2 - eMass**2) # momentum magnitude 

print(mag)

# setting up particles
w = r.TLorentzVector()
#stationary w
w.SetPxPyPzE(0,0,0, wMass)

e = r.TLorentzVector()
n = r.TLorentzVector()

40.149999998443334


In [4]:
def components(num):
    '''given a number of events wanted, returns arrays of the components of the electron and neutrinos
    in the frame of the wboson'''
    
    #initialising arrays for components
    ePx = np.zeros(num)
    ePy = np.zeros(num)
    ePz = np.zeros(num)
    eE = np.zeros(num)
    
    # looping over number of events and filling component arrays
    for i in range(num):
        #making e components
        thetae, phie = thetaPhi()
        vector(e, thetae, phie, mag, wMass/2)
        
        ePx[i] = e.Px()
        ePy[i] = e.Py()
        ePz[i] = e.Pz()
        eE[i] = e.E()
        
    return ePx, ePy, ePz, eE

In [5]:
# making num events worth of components
numberOfEvents = 10
ePx, ePy, ePz, eE = components(numberOfEvents)
wArray =  wArr(numberOfEvents)
wPx, wPy, wPz, wE = wArray[:,0], wArray[:,1], wArray[:,2], wArray[:,3] 

beta = 0.1

#applying the boost to the particles 
wPx, wPy, wPz, wE = boost(wPx, wPy, wPz, wE, beta)
ePx, ePy, ePz, eE = boost(ePx, ePy, ePz, eE, beta)

In [6]:
tree = r.TTree("tree", "initial random e and n components")

wBranch = np.zeros(4)
eBranch = np.zeros(4)

tree.Branch("wBoson",wBranch,"wArray[4]/D")
tree.Branch("electron",eBranch,"elArray[4]/D")

In [7]:
for i in range(len(ePx)):
    wBranch[2] = wPz[i]
    wBranch[3] = wE[i] 
    
    eBranch[0] = ePx[i]
    eBranch[1] = ePy[i]
    eBranch[2] = ePz[i]
    eBranch[3] = eE[i]
   
    tree.Fill()

We can scan the tree to see Values of W boson and electron.

In [8]:
tree.Scan()

40

***********************************************
*    Row   * Instance * wBoson.wB * electron. *
***********************************************
*        0 *        0 *         0 * -8.213797 *
*        0 *        1 *         0 * -1.759209 *
*        0 *        2 * 8.0704536 * -35.42401 *
*        0 *        3 * 80.704536 * 36.406344 *
*        1 *        0 *         0 * -29.09737 *
*        1 *        1 *         0 * 0.0803621 *
*        1 *        2 * 8.0704536 * 31.839719 *
*        1 *        3 * 80.704536 * 43.132717 *
*        2 *        0 *         0 * 7.4278055 *
*        2 *        1 *         0 * 12.739963 *
*        2 *        2 * 8.0704536 * 41.566944 *
*        2 *        3 * 80.704536 * 44.105440 *
*        3 *        0 *         0 * 26.676696 *
*        3 *        1 *         0 * 21.659858 *
*        3 *        2 * 8.0704536 * -16.83539 *
*        3 *        3 * 80.704536 * 38.265206 *
*        4 *        0 *         0 * 16.748908 *
*        4 *        1 *         0 * -33.

Type <CR> to continue or q to quit ==> 

In [9]:
file = r.TFile("generatedComponentsBOOST.root","recreate")
tree.Write()
file.Close